<a href="https://colab.research.google.com/github/roniiiiiii/GenAI_Project/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utils.py

import streamlit as st
import os
import numpy as np
import base64
from io import BytesIO
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

# You will pass `metadata` and `image_data` as arguments where needed

def encode_image_to_base64(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode()

def load_image_and_metadata(metadata, image_id=None, uploaded_file=None):
    if uploaded_file:
        image = Image.open(uploaded_file)
        filename = uploaded_file.name
        meta = None
    elif image_id:
        filename = image_id + ".jpg"
        image_path = os.path.join("images", filename)
        image = Image.open(image_path)
        meta_row = metadata[metadata["id"] + ".jpg" == filename]
        meta = meta_row.iloc[0].to_dict() if not meta_row.empty else None
    else:
        return None, None, None
    return image, filename, meta

def display_metadata(meta):
    if meta:
        st.markdown(f"""
            **Title:** {meta.get('name', 'Unknown')}
            **Artist:** {meta.get('artist', 'Unknown')}
            **Year:** {meta.get('year', 'Unknown')}
            **Nationality:** {meta.get('nationality', 'Unknown')}
        """)
    else:
        st.info("No metadata available for this artwork.")

def build_image_analysis_prompt():
    return (
        "You are a feminist art expert. Carefully analyze the provided artwork. "
        "Focus on visual elements, body language, color palette, composition, symbols, and possible historical or cultural context. "
        "Provide an insightful interpretation of its mood and message."
    )

def get_ai_analysis(client, image):
    base64_img = encode_image_to_base64(image)
    prompt = build_image_analysis_prompt()

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"}}
                ]
            }
        ],
        max_tokens=1024
    )
    return response.choices[0].message.content

def similarity_search(selected_filename, image_data):
    if selected_filename in image_data:
        query_vector = np.array(image_data[selected_filename]["embedding"]).reshape(1, -1)
        results = []
        for filename, data in image_data.items():
            if filename == selected_filename:
                continue
            other_vector = np.array(data["embedding"]).reshape(1, -1)
            sim = cosine_similarity(query_vector, other_vector)[0][0]
            results.append((filename, sim))

        top_results = sorted(results, key=lambda x: x[1], reverse=True)[:3]
        #st.subheader("Visually Similar Artworks")
        cols = st.columns(len(top_results))
        for i, (filename, score) in enumerate(top_results):
            with cols[i]:
                img_path = os.path.join("images", filename)
                if os.path.exists(img_path):
                    st.image(img_path, caption=f"Similarity: {score:.2f}", use_column_width=True)
                else:
                    st.write(f"{filename} not found.")
    else:
        st.info("No precomputed embedding available for this artwork.")

def chat_about_artwork(client, title, artist, year, nationality, ai_analysis):
    st.subheader("Chat About the Artwork")
    system_prompt = (
        f"You are an expert in feminist art. Provide detailed and thoughtful answers.\n\n"
        f"Artwork: {title}\nArtist: {artist}\nYear: {year}\nNationality: {nationality}\n"
        f"AI Analysis: {ai_analysis}"
    )

    if "messages" not in st.session_state or st.session_state.get("current_artwork_title") != title:
        st.session_state.messages = [
            {"role": "system", "content": system_prompt},
            {"role": "assistant", "content": "Ask me anything about this artwork."}
        ]
        st.session_state.current_artwork_title = title

    user_input = st.chat_input("Ask a question about the artwork")
    if user_input:
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                chat_response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=st.session_state.messages,
                    max_tokens=1024
                )
                reply = chat_response.choices[0].message.content
                st.write(reply)
                st.session_state.messages.append({"role": "assistant", "content": reply})

    for msg in st.session_state.messages[1:]:
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])
